# PHP-Parser-Py API Demo

This notebook demonstrates the functionality and APIs of `php-parser-py`, a Python wrapper for PHP-Parser that integrates with cpg2py's graph framework.

## Features

- Parse PHP code into Abstract Syntax Trees (AST)
- Query and traverse AST nodes using graph-based queries
- Access PHP-Parser attributes (line numbers, comments, etc.)
- Generate PHP code from modified ASTs (lossless round-trip)
- Dynamic property access without hardcoded node types

## Setup

First, let's import the library and set up our environment.

In [3]:
import sys
sys.path.insert(0, '../src')

from php_parser_py import parse, parse_file, Parser, PrettyPrinter, AST

## 1. Basic Parsing

Let's start by parsing a simple PHP function.

In [4]:
# Simple PHP code
php_code = """
<?php
function greet($name) {
    echo "Hello, " . $name;
    return true;
}
"""

# Parse the code
ast = parse(php_code)
print(f"Parsed AST with {len(list(ast.nodes()))} nodes")

RunnerError: Failed to execute PHP script: PHP execution failed with exit code 1

## 2. Querying Nodes

The AST provides powerful querying capabilities using cpg2py's graph framework.

In [ ]:
# Find all nodes
all_nodes = list(ast.nodes())
print(f"Total nodes: {len(all_nodes)}\n")

# Find function nodes
functions = list(ast.nodes(lambda n: n.node_type == "Stmt_Function"))
print(f"Found {len(functions)} function(s)")

# Get the first function
if functions:
    func = functions[0]
    print(f"Function type: {func.node_type}")
    print(f"Function name: {func['name']}")

## 3. Accessing Node Properties

Nodes support both Pythonic property access and dict-like access.

In [ ]:
# Get the function node
func = ast.first_node(lambda n: n.node_type == "Stmt_Function")

if func:
    print("=== Pythonic Property Access ===")
    print(f"Node type: {func.node_type}")
    print(f"Start line: {func.start_line}")
    print(f"End line: {func.end_line}")
    print(f"Start file pos: {func.start_file_pos}")
    print(f"End file pos: {func.end_file_pos}")
    
    print("\n=== Dict-like Access ===")
    print(f"Node type: {func['nodeType']}")
    print(f"By reference: {func['byRef']}")
    print(f"Has 'name': {'name' in func}")
    print(f"Return type: {func.get('returnType', 'None')}")
    
    print("\n=== All Properties ===")
    for key, value in func.all_properties.items():
        if not isinstance(value, (dict, list)):
            print(f"  {key}: {value}")

## 4. Traversing the AST

Let's explore the tree structure by finding different node types.

In [ ]:
# Find different types of nodes
node_types = {}
for node in ast.nodes():
    node_type = node.node_type
    if node_type:
        node_types[node_type] = node_types.get(node_type, 0) + 1

print("Node types in the AST:")
for node_type, count in sorted(node_types.items()):
    print(f"  {node_type}: {count}")

## 5. Finding Specific Patterns

Let's find all echo statements and variable usages.

In [ ]:
# Find echo statements
echo_nodes = list(ast.nodes(lambda n: n.node_type == "Stmt_Echo"))
print(f"Found {len(echo_nodes)} echo statement(s)")

# Find variable expressions
var_nodes = list(ast.nodes(lambda n: n.node_type == "Expr_Variable"))
print(f"Found {len(var_nodes)} variable expression(s)")

# Print variable names
print("\nVariable names:")
for var in var_nodes:
    if 'name' in var:
        print(f"  ${var['name']}")

## 6. Code Generation (Round-trip)

One of the key features is lossless round-trip: parse → modify → generate code.

In [ ]:
# Generate PHP code from AST
printer = PrettyPrinter()
generated_code = printer.print(ast)

print("Generated PHP code:")
print("=" * 50)
print(generated_code)
print("=" * 50)

## 7. Working with Complex PHP Code

Let's parse a more complex example with classes and methods.

In [ ]:
complex_php = """
<?php
class User {
    private $name;
    private $email;
    
    public function __construct($name, $email) {
        $this->name = $name;
        $this->email = $email;
    }
    
    public function getName() {
        return $this->name;
    }
}
"""

ast2 = parse(complex_php)

# Find class nodes
classes = list(ast2.nodes(lambda n: n.node_type == "Stmt_Class"))
print(f"Found {len(classes)} class(es)")

if classes:
    cls = classes[0]
    print(f"\nClass name: {cls['name']}")
    print(f"Class at lines {cls.start_line}-{cls.end_line}")

# Find methods
methods = list(ast2.nodes(lambda n: n.node_type == "Stmt_ClassMethod"))
print(f"\nFound {len(methods)} method(s):")
for method in methods:
    print(f"  - {method['name']} (lines {method.start_line}-{method.end_line})")

## 8. Attribute Access

PHP-Parser provides rich metadata through attributes.

In [ ]:
# Get a class method
method = ast2.first_node(lambda n: n.node_type == "Stmt_ClassMethod")

if method:
    print("Method attributes:")
    print(f"  Name: {method['name']}")
    print(f"  Start line: {method.start_line}")
    print(f"  End line: {method.end_line}")
    print(f"  Start token pos: {method.start_token_pos}")
    print(f"  End token pos: {method.end_token_pos}")
    print(f"  Comments: {method.comments}")
    
    # Check attributes
    print(f"\n  Has 'name': {method.has_attribute('name')}")
    print(f"  Has 'returnType': {method.has_attribute('returnType')}")
    print(f"  Get 'flags': {method.get_attribute('flags', 'N/A')}")

## 9. Error Handling

The library provides clear error messages for invalid PHP code.

In [ ]:
from php_parser_py import ParseError

# Try to parse invalid PHP
invalid_php = "<?php function test( { echo 'broken'; }"

try:
    ast3 = parse(invalid_php)
except ParseError as e:
    print(f"Parse error: {e.message}")
    print(f"Line: {e.line}")

## 10. Advanced Querying

Use cpg2py's powerful querying to find complex patterns.

In [ ]:
# Find all function calls
func_calls = list(ast2.nodes(lambda n: n.node_type == "Expr_FuncCall"))
print(f"Found {len(func_calls)} function call(s)")

# Find all property assignments
assignments = list(ast2.nodes(lambda n: n.node_type == "Expr_Assign"))
print(f"Found {len(assignments)} assignment(s)")

# Find all return statements
returns = list(ast2.nodes(lambda n: n.node_type == "Stmt_Return"))
print(f"Found {len(returns)} return statement(s)")

## Summary

This notebook demonstrated:

1. ✅ **Parsing**: Convert PHP code to AST
2. ✅ **Querying**: Find nodes using predicates
3. ✅ **Properties**: Access node data via properties or dict-like syntax
4. ✅ **Traversal**: Navigate the AST structure
5. ✅ **Attributes**: Access PHP-Parser metadata (lines, positions, comments)
6. ✅ **Code Generation**: Lossless round-trip from AST back to PHP
7. ✅ **Error Handling**: Clear error messages for invalid code

### Key Features

- **Dynamic**: No hardcoded node types - all types from PHP-Parser
- **Pythonic**: Properties with snake_case naming
- **Flexible**: Both property and dict-like access
- **Complete**: Full access to PHP-Parser's features
- **Graph-based**: Powered by cpg2py for advanced queries

For more information, see the [README](../README.md) and [design documentation](design.md).